# Lab 3.2 - Kernel trick

**Wykonanie rozwiązań: Marcin Przewięźlikowski**

https://github.com/mprzewie/ml_basics_course

In [ ]:
import sys
sys.path.append("../lab2")

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, KernelPCA
from scipy.io import loadmat
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import os
from dataset_ops import load_dataset, visualize_dataset
from typing import Tuple
from sklearn.decomposition import PCA
%matplotlib inline

## Hiperkula
Wróćmy do hiperkuli wpisanej w hipersześcian z pierwszego zadania. Pokolorujmy  na czubkach narożników sześcianu na czerwono, punkty na krawędziach na żółto (mogą wymagać dodatkowej generacji, bo w losowym rozkładzie się raczej nie pojawią), punkty w jego wnętrzu (ale nie we wnętrzu kuli) na niebiesko, a punkty z kuli na zielono. Schemat kolorów przykładowy, każdy inny który pozwoli odróżnić elementy będzie ok. 

In [ ]:
def in_sphere(points: np.ndarray, R: float=1.0) -> np.ndarray:
    n_dims = points.shape[1]
    center = np.ones(n_dims) * R
    return np.sqrt(((points - center) ** 2).sum(axis=1)) < R

In [ ]:
def cube_points(n_dims: int, n_points: int = 1000, R: float=1.0) -> np.ndarray:
    return np.random.rand(n_points, n_dims) * 2 * R

def vertex_points(n_dims: int, n_points: int = 100, R: float=1.0) -> np.ndarray:
    points = np.random.rand(n_points, n_dims)
    return (points > 1/2).astype(int) * 2 * R

def edge_points(n_dims: int, n_points: int = 100, R: float=1.0) -> np.ndarray:
    points = vertex_points(n_dims, n_points, R)
    indices_to_randomize = np.random.randint(0, n_dims, n_points)
    points[np.arange(n_points), indices_to_randomize] = np.random.rand(n_points) * 2 * R
    return points

def sphere_points(n_dims: int, n_points: int = 1000, R: float=1.0) -> np.ndarray:
    points = cube_points(n_dims, n_points, R)
    return points[in_sphere(points, R)]

def non_sphere_points(n_dims: int, n_points: int = 1000, R: float=1.0) -> np.ndarray:
    points = cube_points(n_dims, n_points, R)
    return points[in_sphere(points, R) == 0]

In [ ]:
def points(
    n_dims: int, 
    n_non_sphere: int=1000,
    n_sphere: int = 1000,
    n_edge: int = 100,
    n_vertex: int = 100,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    non_sphere = non_sphere_points(n_dims, n_non_sphere)
    sphere = sphere_points(n_dims, n_sphere)
    edges = edge_points(n_dims, n_edge)
    vertices = vertex_points(n_dims, n_vertex)
    return non_sphere, sphere, edges, vertices

In [ ]:
colors_list = list(matplotlib.colors.get_named_colors_mapping().keys())
np.random.seed(2)
np.random.shuffle(colors_list)

In [ ]:
non_sphere, sphere, edges, vertices = points(2)

for i, p in enumerate([non_sphere, sphere, edges, vertices]):
    plt.scatter(p[:,0], p[:,1], c=colors_list[i])

Czyli:

niebieskie -kula

szare - wewnątrz sześcianu

czerwone - krawędzie

czarne - wierzchołki

Wykorzystać metodę PCA by wykonać wizualizację (rzut na płaszczyznę 2D) tejże sytuacji dla 3, 4, 5, 7 i 13 wymiarów. 

In [ ]:
dims = [3, 4, 5, 7, 10, 13]

In [ ]:
fig = plt.figure(figsize=(10,15))
fig.suptitle(f"Rzut hipersześcianu na 2D")
for d, n_dims in enumerate(dims):
    non_sphere, sphere, edges, vertices = points(n_dims)
    all_points = np.concatenate([non_sphere, sphere, edges, vertices])
    pca = PCA(n_components=2).fit(all_points)
    plt.subplot(3,2, d+1, title=f"dims = {n_dims}")
    for i, p in enumerate([non_sphere, sphere, edges, vertices]):
        if p.shape[0] > 0:
            p_c = pca.transform(p)
            plt.scatter(p_c[:,0], p_c[:, 1], c=colors_list[i])
plt.show()

Powtórzyć to samo, ale dla wizualizacji 3D. Krótko opisać co widać. ;]

In [ ]:
fig = plt.figure(figsize=(15,25))
fig.suptitle(f"Rzut hipersześcianu na 3D")
for d, n_dims in enumerate(dims):
    non_sphere, sphere, edges, vertices = points(n_dims)
    all_points = np.concatenate([non_sphere, sphere, edges, vertices])
    pca = PCA(n_components=3).fit(all_points)
    ax = fig.add_subplot(3,2,d+1, projection='3d',  title=f"dims = {n_dims}")
    for i, p in enumerate([non_sphere, sphere, edges, vertices]):
        if p.shape[0] > 0:
            p_c = pca.transform(p)
            ax.scatter(xs=p_c[:,0], ys=p_c[:, 1], zs=p_c[:, 2], c=colors_list[i])
    for spine in ax.spines.values():
        spine.set_visible(False)

plt.tight_layout()
plt.show()

### Komentarz

Rzut hiperszescianów na płaszczyzny 2D i 3D wykonany przez PCA przypomina zwykłą wizualizację takiego hipersześcianu - poszczególne punkty znajdują się tam, gdzie można by się ich intuicyjnie spodziewać, widac to zwłaszcza na wizualizacjach sześcianu 3D.

Jak wiemy z Klątwy Wymiarów, prawdopodobieństwo znalezienia w hiperszescianie punktów należących do hiperkuli maleje ze wzrostem wymiarowości, co również jest widoczne na wizulaizacjach.

## Kernel PCA
Wygenerujmy zbiór danych wyglądający +- tak, jak na rysunku w załączniku (punkty w różnych kolorach należą do różnych klas). 

In [ ]:
ds = load_dataset("ds_1.png")
X, y = ds
X_uncentered = X
# scentrowanie datasetu wokół (0,0)
X = X - np.array([50,50])
colors = ['k', 'b', 'r', 'c', 'm', 'y', 'k', 'w']
for c in range(y.max() + 1):
    X_c = X[y==c]
    plt.scatter(X_c[:, 0], X_c[:, 1], color=colors[c], marker=".")
plt.show()

Potraktujmy go klasycznym PCA.

In [ ]:
pca = PCA()
X_t = pca.fit_transform(X)

 Przygotujmy diagram prezentujący rozkład punktów w nowo znalezionej przestrzeni.

In [ ]:
for c in range(y.max() + 1):
    X_c = X_t[y==c]
    plt.scatter(X_c[:, 0], X_c[:, 1], color=colors[c], marker=".")

 Nanieśmy również na pierwotny rysunek strzałki prezentujące wektory będące znalezionymi "principal components" (gdzie długość wektora jest proporcjonalna do wariancji przez niego wyjaśnianej). 

In [ ]:
origin = [[0], [0]]
components = pca.components_ 

plt.quiver(
    *origin, components[:, 0], components[:,1], 
    color=['r', 'g'], angles='xy', scale_units='xy', scale=0.03
)
for c in range(y.max() + 1):
    X_c = X[y==c]
    plt.scatter(X_c[:, 0], X_c[:, 1], color=colors[c], marker=".")


Spoiler - efekty będą dość trywialne (mapowanie z 2D w inne 2D). Można próbować uzyskać więcej stosując tzw. kernel trick - zamiast klasycznego iloczynu skalarnego wykorzysztać inną, odpowiednio spreparowaną funkcję. Zrzutujmy więc oba zbiory w nową przestrzeń, ale tym razem wykorzystując kernel PCA z kernelami:

### cosine 
(tu warto sprawdzić jaki wpływ na wynik będzie miało wcześniejsze wyśrodkowanie danych lub jego brak) 

In [ ]:
fig = plt.figure(figsize=(15, 30))
fig.suptitle("Cosine kernel PCA with dataset offset")
for i, offset in enumerate([0] + [2**i for i in range(9)]):
    cosine_pca = KernelPCA(n_components=2, kernel="cosine")
    offset_arr = np.array([offset, offset])
    X_offset = X + offset_arr
    X_t = cosine_pca.fit_transform(X_offset)
    origin = [[0], [0]]
    components = cosine_pca.alphas_
    plt.subplot(5,2, i+1)
    plt.title(f"offset = {offset_arr}")
    colors = ['k', 'b', 'r', 'c', 'm', 'y', 'k', 'w']
    for c in range(y.max() + 1):
        X_c = X_t[y==c]
        plt.scatter(X_c[:, 0], X_c[:, 1], color=colors[c], marker=".")
        plt.quiver(
            *origin, components[y==c, 0], components[y==c,1], 
            color=colors[c],
            angles='xy', 
            scale_units='xy', 
            scale=0.2
        )
plt.show()

#### Komentarz

Rozstawienie punktów w transformacji Cosine kernel PCA jest zależne od kąta, pod którym leżą te punkty od punktu $(0,0)$ w oryginalnym datasecie.

Jeżeli wytrenujemy Cosine kernel PCA na zbiorze danych skoncentrowanym wokół $(0,0)$, transformacja tego zbioru będzie wyglądac jak okrag, natomiast ze zwiększaniem odległości "centrum" datasetu od $(0,0)$, większość punktów trafia na coraz mniejszy wycinek "okręgu", a tylko niektóre w inne jego miejsca.

### rbf 
(radial basis function - tu należy sprawdzić różne wartości parametru gamma wpływającego na pracę kernela, w tym także bardzo małe) oraz przygotujmy diagramy prezentujące efekty.

In [ ]:
fig = plt.figure(figsize=(15, 30))
fig.suptitle("RBF kernel PCA")
for i, gamma in enumerate([2 ** i / (2**10) for i in range(10)]):
    rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=gamma)
    X_t = rbf_pca.fit_transform(X)
    origin = [[0], [0]]
    components = rbf_pca.alphas_
    plt.subplot(5,2, i+1)
    plt.title(f"gamma = {gamma}")
    colors = ['k', 'b', 'r', 'c', 'm', 'y', 'k', 'w']
    for c in range(y.max() + 1):
        X_c = X_t[y==c]
        plt.scatter(X_c[:, 0], X_c[:, 1], color=colors[c], marker=".")
        plt.quiver(
            *origin, components[y==c, 0], components[y==c,1], 
            color=colors[c],
            angles='xy', 
            scale_units='xy', 
            scale=0.5
        )
plt.show()

#### Komentarz

RBF grupuje punkty o podobnej odległości od środka (średniej) zbioru danych. Wraz ze wzrostem parametru $\gamma$, zmniejsza się rozrzut kierunków otrzymanych komponentów. Warto zauważyc, że jest to całkiem dobra transformacja naszego datasetu, gdyż w jego przypadku punkty tych samych klas leżą zazwyczaj w określonych odległościach od centrum datasetu.